https://www.geeksforgeeks.org/ml-handling-imbalanced-data-with-smote-and-near-miss-algorithm-in-python/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
!pip install imbalanced-learn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz

In [31]:
#  Read the CSV and Perform Basic Data Cleaning
data = pd.read_csv('../data/combined_wine.csv', sep=',')
# Drop the null columns where all values are null
data = data.dropna(axis='columns', how='all')
# Drop the null rows
data = data.dropna()

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [32]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 710.6 KB
None


In [33]:
# normalise the quality column 
data['normquality'] = StandardScaler().fit_transform(np.array(data['quality']).reshape(-1, 1)) 
# drop irrevelent columns as they are not relevant for prediction purpose  
# df = df.drop(['fixed acidity', 'citric acid', 'residual sugar', 'free sulfur dioxide', 'total sulfur dioxide', 'pH', 'sulphates'], axis = 1) 
  
# as you can see there are 492 fraud transactions. 
data['type'].value_counts() 

0    4898
1    1599
Name: type, dtype: int64

In [34]:
X = data.drop(columns=["quality"])
y = data["quality"]
print(X.shape, y.shape)

(6497, 13) (6497,)


In [35]:
# Let's split the data into training and testing sets:
X_train = data.values[0:5000]
X_test = data.values[5000:]

In [36]:
from sklearn.model_selection import train_test_split

# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# describes info about train and test set
print("X_train dataset: ", X_train.shape) 
print("y_train dataset: ", y_train.shape) 
print("X_test dataset: ", X_test.shape) 
print("y_test dataset: ", y_test.shape) 

X_train dataset:  (4547, 13)
y_train dataset:  (4547,)
X_test dataset:  (1950, 13)
y_test dataset:  (1950,)


In [37]:
# logistic regression object 
lr = LogisticRegression() 
  
# train the model on train set 
lr.fit(X_train, y_train.ravel()) 
  
predictions = lr.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

           3       0.00      0.00      0.00        12
           4       0.00      0.00      0.00        64
           5       0.85      0.94      0.89       643
           6       0.83      0.96      0.89       845
           7       0.75      0.56      0.64       329
           8       0.00      0.00      0.00        54
           9       0.00      0.00      0.00         3

    accuracy                           0.82      1950
   macro avg       0.35      0.35      0.35      1950
weighted avg       0.77      0.82      0.79      1950



C:\Users\ptsai\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ptsai\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
!pip install mlrose

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=898dc9853637cdab0e20cce5b10e93df3ea60963f08a05ab9ce3292030cae517
  Stored in directory: c:\users\ptsai\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [38]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 
  
# import SMOTE module from imblearn library 
#!pip install mlrose (done in cell above), six
#!pip instal imblearn done in first cell
from imblearn.over_sampling import SMOTE
import six
import sys
sys.modules['sklearn.externals.six'] = six
from six import StringIO
import mlrose

sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 

Before OverSampling, counts of label '1': 0
Before OverSampling, counts of label '0': 0 



ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 6

In [31]:
lr1 = LogisticRegression() 
lr1.fit(X_train_res, y_train_res.ravel()) 
predictions = lr1.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, predictions)) 

NameError: name 'X_train_res' is not defined